In [17]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Annotated

In [18]:
def reducer_func(old: list[str], new: list[str]):
    return old + new

In [ ]:
import operator


class State(TypedDict):
    messages: Annotated[list[str], reducer_func]
    # messages: Annotated[list[str], operator.add]


graph_builder = StateGraph(
    State,
)

In [20]:
def node_one(state: State) -> State:
    messages = state.get("messages")
    last_message = messages[-1]
    response = "Hello, nice to meet you"

    return {"messages": [response]}


def node_two(state: State):

    return {}


def node_three(state: State) -> State:

    return {}

In [21]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)
graph_builder.add_node("node_three", node_three)


graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", END)

In [22]:
a: State = {"messages": ["Hello!"]}

graph = graph_builder.compile()

graph.invoke(a)

{'messages': ['Hello!', 'Hello, nice to meet you']}